In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
records = [("SV001", "Anh", "Toán", 85), ("SV001", "Anh", "Vật Lý", 90), ("SV001", "Anh", "Hóa Học", 75),
          ("SV002", "Bình", "Toán", 78), ("SV002", "Bình", "Vật Lý", 82), ("SV002", "Bình", "Hóa Học", 88),
          ("SV003", "Cường", "Toán", 92), ("SV003", "Cường", "Vật Lý", 87), ("SV003", "Cường", "Hóa Học", 94)]

In [5]:
rdd = spark.sparkContext.parallelize(records)

In [6]:
df = rdd.toDF()

In [7]:
cols = ["Mã SV", "Tên SV", "Môn Học", "Điểm"]
df = df.toDF(*cols)
df.show()

+-----+------+-------+----+
|Mã SV|Tên SV|Môn Học|Điểm|
+-----+------+-------+----+
|SV001|   Anh|   Toán|  85|
|SV001|   Anh| Vật Lý|  90|
|SV001|   Anh|Hóa Học|  75|
|SV002|  Bình|   Toán|  78|
|SV002|  Bình| Vật Lý|  82|
|SV002|  Bình|Hóa Học|  88|
|SV003| Cường|   Toán|  92|
|SV003| Cường| Vật Lý|  87|
|SV003| Cường|Hóa Học|  94|
+-----+------+-------+----+



In [8]:
# df.write.csv('pivotTest.csv') Cái này export ra 1 folder

In [9]:
pandasDF = df.toPandas()

In [10]:
pandasDF.to_csv('pivotTest.csv', index=False)

In [11]:
df.groupBy('Mã SV').pivot('Môn Học').sum('Điểm').show()

+-----+-------+----+------+
|Mã SV|Hóa Học|Toán|Vật Lý|
+-----+-------+----+------+
|SV002|     88|  78|    82|
|SV003|     94|  92|    87|
|SV001|     75|  85|    90|
+-----+-------+----+------+



In [12]:
records2 = [("SP001", "Điện Thoại", 100, "Hà Nội"), ("SP002", "Laptop", 50, "Hà Nội"),
           ("SP003", "Máy Tính", 80, "TPHCM"), ("SP001", "Điện Thoại", 120, "TPHCM"),
           ("SP002", "Laptop", 70, "TPHCM"), ("SP003", "Máy Tính", 90, "Hà Nội")]

In [13]:
rdd2 = spark.sparkContext.parallelize(records2)

In [14]:
cols2 = ["Mã SP", "Loại SP", "Doanh số", "Khu vực"]

In [15]:
df2 = rdd2.toDF().toDF(*cols2)

In [16]:
df2.show()

+-----+----------+--------+-------+
|Mã SP|   Loại SP|Doanh số|Khu vực|
+-----+----------+--------+-------+
|SP001|Điện Thoại|     100| Hà Nội|
|SP002|    Laptop|      50| Hà Nội|
|SP003|  Máy Tính|      80|  TPHCM|
|SP001|Điện Thoại|     120|  TPHCM|
|SP002|    Laptop|      70|  TPHCM|
|SP003|  Máy Tính|      90| Hà Nội|
+-----+----------+--------+-------+



In [17]:
df2.groupBy("Khu vực").pivot("Loại SP").sum("Doanh số").show()

+-------+------+--------+----------+
|Khu vực|Laptop|Máy Tính|Điện Thoại|
+-------+------+--------+----------+
|  TPHCM|    70|      80|       120|
| Hà Nội|    50|      90|       100|
+-------+------+--------+----------+



In [19]:
tempdf = df.groupBy('Mã SV').pivot('Môn Học').sum('Điểm')

In [20]:
tempdf2 = df2.groupBy("Khu vực").pivot("Loại SP").sum("Doanh số")

In [22]:
merged_df = tempdf2.union(tempdf)
merged_df.show()

+-------+------+--------+----------+
|Khu vực|Laptop|Máy Tính|Điện Thoại|
+-------+------+--------+----------+
|  TPHCM|    70|      80|       120|
| Hà Nội|    50|      90|       100|
|  SV002|    88|      78|        82|
|  SV003|    94|      92|        87|
|  SV001|    75|      85|        90|
+-------+------+--------+----------+



In [23]:
records3 = [('KH001', 35, 'Nam', 3), ('KH002', 28, 'Nữ', 2), ('KH003', 40, 'Nam', 4)
           ,('KH004', 45, 'Nữ', 1), ('KH005', 30, 'Nam', 5),('KH006', 35, 'Nữ', 2)]

In [24]:
rdd3 = spark.sparkContext.parallelize(records3)

In [26]:
cols3 = ("Mã khách hàng", "Độ tuổi", "Giới tính", "Số lần mua hàng")

In [27]:
df3 = rdd3.toDF().toDF(*cols3)

In [28]:
df3.show()

+-------------+-------+---------+---------------+
|Mã khách hàng|Độ tuổi|Giới tính|Số lần mua hàng|
+-------------+-------+---------+---------------+
|        KH001|     35|      Nam|              3|
|        KH002|     28|       Nữ|              2|
|        KH003|     40|      Nam|              4|
|        KH004|     45|       Nữ|              1|
|        KH005|     30|      Nam|              5|
|        KH006|     35|       Nữ|              2|
+-------------+-------+---------+---------------+



In [37]:
from datetime import datetime

In [43]:
records4 = [('DH001', datetime.strptime('2024-03-01', '%Y-%m-%d').date(), 'Điện Thoại', 10)
           ,('DH002', datetime.strptime('2024-03-01', '%Y-%m-%d').date(), 'Laptop', 5)
           ,('DH003', datetime.strptime('2024-03-02', '%Y-%m-%d').date(), 'Máy tính', 8)
           ,('DH004', datetime.strptime('2024-03-03', '%Y-%m-%d').date(), 'Điện Thoại', 12)
           ,('DH005', datetime.strptime('2024-03-03', '%Y-%m-%d').date(), 'Laptop', 7)
           ,('DH006', datetime.strptime('2024-03-04', '%Y-%m-%d').date(), 'Máy tính', 9)]

In [44]:
cols4 = ('Mã đơn hàng', 'Ngày đặt hàng', 'Sản phẩm', 'Số lượng')

In [45]:
rdd4 = spark.sparkContext.parallelize(records4)

In [46]:
df4 = rdd4.toDF().toDF(*cols4)

In [47]:
df4.show()

+-----------+-------------+----------+--------+
|Mã đơn hàng|Ngày đặt hàng|  Sản phẩm|Số lượng|
+-----------+-------------+----------+--------+
|      DH001|   2024-03-01|Điện Thoại|      10|
|      DH002|   2024-03-01|    Laptop|       5|
|      DH003|   2024-03-02|  Máy tính|       8|
|      DH004|   2024-03-03|Điện Thoại|      12|
|      DH005|   2024-03-03|    Laptop|       7|
|      DH006|   2024-03-04|  Máy tính|       9|
+-----------+-------------+----------+--------+



In [54]:
records5 = [('Nguyễn Văn A', 'Kế Toán', datetime.strptime('2020-01-01', '%Y-%m-%d').date(), 800)
           ,('Trần Thị B', 'Kinh Doanh', datetime.strptime('2019-05-01', '%Y-%m-%d').date(), 900)
           ,('Lê Văn C', 'Kỹ Thuật', datetime.strptime('2021-03-15', '%Y-%m-%d').date(), 1000)
           ,('Hoàng Thị D', 'Nhân Sự', datetime.strptime('2022-07-01', '%Y-%m-%d').date(), 850)
           ,('Phạm Văn E', 'Marketing', datetime.strptime('2023-02-01', '%Y-%m-%d').date(), 950)
           ,('Đồ Thị F', 'Kỹ Thuật', datetime.strptime('2020-10-01', '%Y-%m-%d').date(), 1100)]

In [55]:
cols5 = ('Tên nhân viên', 'Phòng ban', 'Ngày bắt đầu', 'Lương')

In [56]:
rdd5 = spark.sparkContext.parallelize(records5)

In [57]:
df5 = rdd5.toDF().toDF(*cols5)

In [58]:
df5.show()

+-------------+----------+------------+-----+
|Tên nhân viên| Phòng ban|Ngày bắt đầu|Lương|
+-------------+----------+------------+-----+
| Nguyễn Văn A|   Kế Toán|  2020-01-01|  800|
|   Trần Thị B|Kinh Doanh|  2019-05-01|  900|
|     Lê Văn C|  Kỹ Thuật|  2021-03-15| 1000|
|  Hoàng Thị D|   Nhân Sự|  2022-07-01|  850|
|   Phạm Văn E| Marketing|  2023-02-01|  950|
|     Đồ Thị F|  Kỹ Thuật|  2020-10-01| 1100|
+-------------+----------+------------+-----+



In [60]:
df3.groupBy('Độ tuổi').pivot('Giới tính').sum('Số lần mua hàng').show()

+-------+----+----+
|Độ tuổi| Nam|  Nữ|
+-------+----+----+
|     28|NULL|   2|
|     35|   3|   2|
|     30|   5|NULL|
|     40|   4|NULL|
|     45|NULL|   1|
+-------+----+----+



In [96]:
from pyspark.sql.functions import year, dayofweek

In [97]:
df4 = df4.withColumn("Ngày trong tuần", dayofweek("Ngày đặt hàng"))

In [101]:
df4.groupBy('Sản phẩm').pivot('Ngày trong tuần').sum('Số lượng').show()

+----------+----+----+----+----+
|  Sản phẩm|   1|   2|   6|   7|
+----------+----+----+----+----+
|    Laptop|   7|NULL|   5|NULL|
|Điện Thoại|  12|NULL|  10|NULL|
|  Máy tính|NULL|   9|NULL|   8|
+----------+----+----+----+----+



In [99]:
df5 = df5.withColumn("Năm bắt đầu", year("Ngày bắt đầu"))

In [100]:
df5.groupBy('Phòng ban').pivot('Năm bắt đầu').sum('Lương').show()

+----------+----+----+----+----+----+
| Phòng ban|2019|2020|2021|2022|2023|
+----------+----+----+----+----+----+
|   Nhân Sự|NULL|NULL|NULL| 850|NULL|
|  Kỹ Thuật|NULL|1100|1000|NULL|NULL|
|Kinh Doanh| 900|NULL|NULL|NULL|NULL|
|   Kế Toán|NULL| 800|NULL|NULL|NULL|
| Marketing|NULL|NULL|NULL|NULL| 950|
+----------+----+----+----+----+----+

